### Take Home Assignement

This **notebook** is considered as the same time as the main & README to learn how to use the package.

They is 4 different steps in the project:
1. Introduction
2. Create the Fake Database & Tables.
3. Generate the fake Dataset.
4. Report 3 relevant KPIs.


- ```git clone git@github.com:MatthieuRu/take-home-assignement.git``` *Git clone the project from Github.*
- ```./start.sh``` *Launch the docker images including the postgre server*
- ```conda env create``` *Create the conda environment based on the environment.yml.*
- ```conda activate assigenment``` *Activate the conda environment.*
- ```pytest src -v -s``` *Run the unit tests.*
- ```jupyter notebook README.ypnb``` *You can copy paste the README.ipynb and start to play with the package*

### 1. Introduction

Before to start the project, the stakeholders of these project has discussed about the expected output of the package:
- The Data Analyst & Engineer is playing the role of *Product Owner* and *Development Team*.
- The Leadership team is playing the *client*'s role. 

After the discussion, the ouptut of the KPI should cover some topics:
1. The Company's Revenue
2. The Client's Retention
3. The Product's Usage


In this context, a list of  3 KPI / 3 user stories has been approved by both team in term of getting all the information needed by the Leadership team:


**List of User story:**
1. As a a menber of the Leadership team, I want to know how many client group by company size at the month X.
1. As a a menber of the Leadership team, I want to how much is the revenu group by company size at he month X.
1. As a a menber of the Leadership team, I want to how many time our customers has used in average our product group by company size at he month X.



### 2. DB Management

The main goal of this database is to not overload the database with useless information. If we need to add a column or a table, it's because we cannot achieve a user story with the actual database structure.



<img src='schemas.png' width="500" height="200">

In [1]:
from src.main.assignment.db_manager.server import Server
# Connection to the server
server = Server(
    ip='0.0.0.0',
    user='application',
    passwd='secretpassword',
    database='application'
) 

In [ ]:
# Create the schema and the needed tables.
server._create_schema_abc('./src/main/assignment/db_manager/ABC.sql')

In [ ]:
# Delete the schema (if needed)
server._delete_schema_abc()

### 3. Generate Fake Table

**Rules**
 - Let's assume the company has existed for 12 months, they acquired all their customers on month 1 and they will not gain any more customers in the following 12 months.
 - All customers (referred to as companies) have monthly subscriptions.
 - Create 500 companies, each with a unique `company_id` .
 - Each company is a company_size="small" with a probability of 0. 7 and "large" with a probability of 0. 3.
 - Each small company has the number of sessions in a month equal to 5, plus or minus a uniform random integer between 0 & 5.
 - Each large company has the number of sessions in a month equal to 10, plus or minus a uniform random integer between 0 & 10.
 - Give each session a random `created_at` timestamp within that month.
 - If a company has 0 sessions in a month, they have churned that month and will not come back (i. e. have no future sessions).

**Step A. Initate the fake data generation:**
1. Create the company table for the month 0

**Step B. Implement all the followins month iteration:**
1. Filter the company still active on the product by using session_information month X-1
2. Create the session_information table for the month X


In [11]:
from src.main.assignment.db_manager.generator import Generator
import datetime

starting_date = datetime.date(2021, 1, 1)
nb_company = 500
generator = Generator(
    starting_date,
    nb_company,
    nb_month=20
)

In [ ]:
# send the fake data into the database.
generator.send_to_database(server)

In [ ]:
# Delete the data (if needed)
generator.delete_from_database(server)

### 4. Report 3 relevant KPIs

The KPI is following the **scope / design** aligned at the introduction of the project.

The idea is to be able to track overtime all the relevant topics and group by the company size.

In this case, all the stakeholder has all the information needed to get:
1. An overview of the health of the company
2. Inititiate a deeper analysis (eg compare two periods - concatenate the two type of company - get number of lost company)



In [13]:
from src.main.assignment.db_manager.get_kpi import get_kpi_usage

get_kpi_usage(server)

,period,company_size,nb_average_session
0,2021-01-01,large,9
1,2021-01-01,small,5
2,2021-02-01,large,9
3,2021-02-01,small,5
4,2021-03-01,large,10
5,2021-03-01,small,5
6,2021-04-01,large,10
7,2021-04-01,small,5
8,2021-05-01,large,11
9,2021-05-01,small,4


In [14]:
from src.main.assignment.db_manager.get_kpi import get_kpi_retention

get_kpi_retention(server)

,period,nb_company,company_size
0,2021-01-01,141,large
1,2021-01-01,309,small
2,2021-02-01,131,large
3,2021-02-01,270,small
4,2021-03-01,125,large
5,2021-03-01,241,small
6,2021-04-01,124,large
7,2021-04-01,216,small
8,2021-05-01,116,large
9,2021-05-01,189,small


In [15]:
from src.main.assignment.db_manager.get_kpi import get_kpi_revenue

get_kpi_revenue(server)

,period,revenu,company_size
0,2021-01-01,13959,large
1,2021-02-01,12969,large
2,2021-03-01,12375,large
3,2021-04-01,12276,large
4,2021-05-01,11484,large
5,2021-06-01,10890,large
6,2021-07-01,10197,large
7,2021-08-01,9603,large
8,2021-09-01,9405,large
9,2021-10-01,8811,large
